In [1]:
import pandas as pd 
import numpy as np 


In [2]:
#Load DataA.csv 

dataA = pd.read_csv('~/Documents/Uwaterloo_Study_Docs/657A_ECE/Assignments/ece_657a_assignments/Assignment_1/Datasets/DataA.csv')

In [3]:
print("dataA: \n", dataA)

dataA: 
        Unnamed: 0  fea.1  fea.2  fea.3  fea.4   fea.5  fea.6  fea.7  fea.8  \
0               1 -153.0  414.0  939.0 -161.0  1007.0   99.0 -210.0  948.0   
1               2 -150.0  420.0  939.0 -177.0  1008.0  103.0 -207.0  939.0   
2               3 -160.0  432.0  941.0 -162.0   982.0   98.0 -198.0  936.0   
3               4 -171.0  432.0  911.0 -174.0   999.0  115.0 -187.0  918.0   
4               5 -171.0    NaN  929.0 -189.0  1004.0  104.0 -198.0  939.0   
...           ...    ...    ...    ...    ...     ...    ...    ...    ...   
18995       18996    NaN    NaN    NaN    NaN     NaN    NaN    NaN    NaN   
18996       18997    NaN    NaN    NaN    NaN     NaN    NaN    NaN    NaN   
18997       18998    NaN    NaN    NaN    NaN     NaN    NaN    NaN    NaN   
18998       18999    NaN    NaN    NaN    NaN     NaN    NaN    NaN    NaN   
18999       19000    NaN    NaN    NaN    NaN     NaN    NaN    NaN    NaN   

       fea.9  ...  fea.72  fea.73  fea.74  fea.75  fea

In [4]:
# Display all columns
pd.set_option('display.max_columns', None)

## Shape of the Dataset

In [5]:
dataA.shape

(19000, 82)

#### The Dataset has 19000 samples and 82 features. However there is a Unamed column 'Unnamed' which is just the count which needs to be removed from the dataset 

## Checking all the Columns

In [6]:
dataA.columns

Index(['Unnamed: 0', 'fea.1', 'fea.2', 'fea.3', 'fea.4', 'fea.5', 'fea.6',
       'fea.7', 'fea.8', 'fea.9', 'fea.10', 'fea.11', 'fea.12', 'fea.13',
       'fea.14', 'fea.15', 'fea.16', 'fea.17', 'fea.18', 'fea.19', 'fea.20',
       'fea.21', 'fea.22', 'fea.23', 'fea.24', 'fea.25', 'fea.26', 'fea.27',
       'fea.28', 'fea.29', 'fea.30', 'fea.31', 'fea.32', 'fea.33', 'fea.34',
       'fea.35', 'fea.36', 'fea.37', 'fea.38', 'fea.39', 'fea.40', 'fea.41',
       'fea.42', 'fea.43', 'fea.44', 'fea.45', 'fea.46', 'fea.47', 'fea.48',
       'fea.49', 'fea.50', 'fea.51', 'fea.52', 'fea.53', 'fea.54', 'fea.55',
       'fea.56', 'fea.57', 'fea.58', 'fea.59', 'fea.60', 'fea.61', 'fea.62',
       'fea.63', 'fea.64', 'fea.65', 'fea.66', 'fea.67', 'fea.68', 'fea.69',
       'fea.70', 'fea.71', 'fea.72', 'fea.73', 'fea.74', 'fea.75', 'fea.76',
       'fea.77', 'fea.78', 'fea.79', 'fea.80', 'fea.81'],
      dtype='object')

#### Removing 'Unnamed' columns from the dataset  

In [7]:
dataA_updated = dataA.drop(['Unnamed: 0'], axis=1)
print("Updated Dataset after removal of 'Unnamed: 0' column: \n", dataA_updated.head(20))

Updated Dataset after removal of 'Unnamed: 0' column: 
     fea.1  fea.2  fea.3  fea.4   fea.5  fea.6  fea.7   fea.8  fea.9  fea.10  \
0  -153.0  414.0  939.0 -161.0  1007.0   99.0 -210.0   948.0  333.0   -19.0   
1  -150.0  420.0  939.0 -177.0  1008.0  103.0 -207.0   939.0  316.0     9.0   
2  -160.0  432.0  941.0 -162.0   982.0   98.0 -198.0   936.0  315.0   -10.0   
3  -171.0  432.0  911.0 -174.0   999.0  115.0 -187.0   918.0  338.0    34.0   
4  -171.0    NaN  929.0 -189.0  1004.0  104.0 -198.0   939.0  350.0    60.0   
5  -171.0  432.0  924.0 -179.0  1011.0   85.0 -204.0   945.0  336.0    94.0   
6  -169.0  429.0  949.0 -175.0  1007.0  102.0 -188.0   914.0  322.0   154.0   
7  -160.0  423.0  927.0 -195.0   996.0  123.0 -213.0   925.0  302.0   128.0   
8  -163.0  432.0  929.0 -178.0   994.0  101.0 -186.0   946.0  296.0   166.0   
9  -156.0  415.0  936.0 -186.0  1014.0  111.0 -195.0   960.0  280.0   202.0   
10 -153.0  413.0  923.0 -187.0   993.0   91.0 -193.0   970.0  282.0   233.0

In [8]:
dataA_updated.shape

(19000, 81)

#### The Column 'Unnamed: 0' has been removed from the dataset as we can see in the above data and now the dimension of the dataset has become (19000, 81) which means it has 19000 samples and 81 features in total

## Identifying Missing Values in the Dataset 


In [9]:
dataA_updated.isnull()

,fea.1,fea.2,fea.3,fea.4,fea.5,fea.6,fea.7,fea.8,fea.9,fea.10,fea.11,fea.12,fea.13,fea.14,fea.15,fea.16,fea.17,fea.18,fea.19,fea.20,fea.21,fea.22,fea.23,fea.24,fea.25,fea.26,fea.27,fea.28,fea.29,fea.30,fea.31,fea.32,fea.33,fea.34,fea.35,fea.36,fea.37,fea.38,fea.39,fea.40,fea.41,fea.42,fea.43,fea.44,fea.45,fea.46,fea.47,fea.48,fea.49,fea.50,fea.51,fea.52,fea.53,fea.54,fea.55,fea.56,fea.57,fea.58,fea.59,fea.60,fea.61,fea.62,fea.63,fea.64,fea.65,fea.66,fea.67,fea.68,fea.69,fea.70,fea.71,fea.72,fea.73,fea.74,fea.75,fea.76,fea.77,fea.78,fea.79,fea.80,fea.81
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18995,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
18996,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,Tr

#### Here We can see in the dataset there are many missing value in the Dataset 

## Number of Missing values per columns

In [10]:
# Find the Total number of missing values in each column
dataA_updated.isnull().sum()

fea.1     1187
fea.2     1188
fea.3     1187
fea.4      800
fea.5      800
          ... 
fea.77     773
fea.78     773
fea.79     773
fea.80     773
fea.81     773
Length: 81, dtype: int64

#### As we can see from the above table that the dataset contains too many missing values and each feature has different number of missing values. As from the table it is clear that feature.1 has 1187 missing values, feature.2 has 1188 missing values while from feature.37 to feature.81 all have a missing value of 773 which is the lowest value missing among all the features. Whereas feature.34 to feature.36 have 18999 missing values which highest among all the features in the entire dataset. 

## Total number of Missing values in the dataset

In [11]:
dataA_updated.isnull().sum().sum()

124053

#### In total there are 124053 missing values in the dataset

## Statistical Description of the Dataset

In [12]:
dataA_updated.describe()

,fea.1,fea.2,fea.3,fea.4,fea.5,fea.6,fea.7,fea.8,fea.9,fea.10,fea.11,fea.12,fea.13,fea.14,fea.15,fea.16,fea.17,fea.18,fea.19,fea.20,fea.21,fea.22,fea.23,fea.24,fea.25,fea.26,fea.27,fea.28,fea.29,fea.30,fea.31,fea.32,fea.33,fea.34,fea.35,fea.36,fea.37,fea.38,fea.39,fea.40,fea.41,fea.42,fea.43,fea.44,fea.45,fea.46,fea.47,fea.48,fea.49,fea.50,fea.51,fea.52,fea.53,fea.54,fea.55,fea.56,fea.57,fea.58,fea.59,fea.60,fea.61,fea.62,fea.63,fea.64,fea.65,fea.66,fea.67,fea.68,fea.69,fea.70,fea.71,fea.72,fea.73,fea.74,fea.75,fea.76,fea.77,fea.78,fea.79,fea.80,fea.81
count,17813.000000,17812.000000,17813.000000,18200.000000,18200.000000,18200.000000,18099.000000,18099.000000,18099.000000,18043.000000,18044.000000,18044.000000,17950.000000,17950.000000,17950.000000,18202.000000,18202.000000,18202.000000,17964.000000,17964.000000,17964.000000,17677.000000,17677.000000,17677.000000,18106.000000,18106.000000,18106.000000,18106.000000,18106.000000,18106.000000,18083.000000,18083.000000,18083.000000,1.0,1.0,1.0,18227.000000,18227.000000,18227.000000,18227.00000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.00000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18226.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000,18227.000000
mean,-132.812384,698.264485,597.541402,-307.128462,909.548077,-32.760824,61.974363,899.313498,81.650478,356.638752,330.971569,584.833130,583.589861,421.624290,-38.719833,71.807768,780.358257,-461.605043,274.476843,893.086451,-89.299933,-622.139334,233.764044,-131.400068,-360.661825,875.943278,269.614934,-570.500166,432.334420,427.547277,-124.834707,369.141625,646.348062,0.0,0.0,0.0,-862.982169,-209.995885,209.005047,-0.55736,-7.673890,-9.025347,489.460306,83.230153,209.654194,330.984199,-66.071981,290.883086,6.278762,-817.288473,140.022110,294.320568,13.456027,-9.604268,-8.376420,279.558786,-267.420091,-105.234158,204.50107,-135.484611,197.207714,299.903166,-200.212322,555.226532,443.874637,1.923355,1.402535,-3.169364,-70.556537,-90.936080,-227.353157,-124.658035,-37.973391,137.400176,374.762934,-880.583804,-47.607780,137.641192,-18.099523,4.671257,20.726834
std,284.183187,375.672475,396.654659,183.151634,193.963300,254.001018,317.393784,196.829252,327.904371,343.131382,543.517364,282.078053,307.610433,543.293018,495.913749,258.937246,291.642323,261.243774,298.191766,216.179301,221.071614,359.196753,587.480524,460.080469,223.329186,253.761954,371.407492,307.329729,497.493607,204.477583,369.363351,523.492523,287.282436,NaN,NaN,NaN,204.443034,252.273113,288.117323,622.00637,328.654972,248.500906,344.436164,366.809464,552.049211,521.513609,322.307191,452.056841,470.386277,240.623758,409.226342,229.603664,838.521518,518.143698,560.794366,396.007945,550.994437,385.236647,460.46979,337.120625,421.161877,554.948504,573.494633,339.780793,298.892877,1020.487633,700.587030,867.141376,580.008611,480.162249,379.058945,481.492994,355.841529,352.788441,583.792739,217.634117,373.064609,248.988603,778.015520,480.779966,455.160604
min,-2724.000000,-855.000000,-2196.000000,-1365.000000,-245.000000,-920.000000,-1580.000000,-149.000000,-1624.000000,-1792.000000,-1545.000000,-1079.000000,-1710.000000,-1120.000000,-1634.000000,-1089.000000,-289.000000,-1379.000000,-2269.000000,-786.000000,-1944.000000,-3172.000000,-2324.000000,-2734.000000,-1873.000000,-442.000000,-1430.000000,-1933.000000,-1103.000000,-765.000000,-1717.000000,-1234.000000,-784.000000,0.0,0.0,0.0,-1578.000000,-887.000000,-1067.000000,-3322.00000,-2764.000000,-1293.000000,-767.000000,-985.000000,-975.000000,-876.000000,-705.000000,-702.000000,-983.000000,-1884.000000,-1516.000000,-1671.000000,-12310.000000,-5226.000000,-3541

#### As we can see from the above statistical description of the dataset it clear that the rows of feature.34, feature.35 and feature.36 is entirely missing so there is no statistical value for those columns. We have to remove the entire feature.34, feature.35 and feature.36 from the dataset.

## Drop feature.34, feature.35 and feature.36

In [13]:
dataA_updated = dataA_updated.drop(['fea.34', 'fea.35', 'fea.36'], axis=1)

In [14]:
dataA_updated

,fea.1,fea.2,fea.3,fea.4,fea.5,fea.6,fea.7,fea.8,fea.9,fea.10,fea.11,fea.12,fea.13,fea.14,fea.15,fea.16,fea.17,fea.18,fea.19,fea.20,fea.21,fea.22,fea.23,fea.24,fea.25,fea.26,fea.27,fea.28,fea.29,fea.30,fea.31,fea.32,fea.33,fea.37,fea.38,fea.39,fea.40,fea.41,fea.42,fea.43,fea.44,fea.45,fea.46,fea.47,fea.48,fea.49,fea.50,fea.51,fea.52,fea.53,fea.54,fea.55,fea.56,fea.57,fea.58,fea.59,fea.60,fea.61,fea.62,fea.63,fea.64,fea.65,fea.66,fea.67,fea.68,fea.69,fea.70,fea.71,fea.72,fea.73,fea.74,fea.75,fea.76,fea.77,fea.78,fea.79,fea.80,fea.81
0,-153.0,414.0,939.0,-161.0,1007.0,99.0,-210.0,948.0,333.0,-19.0,-587.0,810.0,902.0,-140.0,-468.0,-28.0,1016.0,-191.0,358.0,895.0,-189.0,-511.0,-782.0,-212.0,-241.0,472.0,938.0,-882.0,-61.0,543.0,141.0,-150.0,965.0,-989.0,-158.0,41.0,78.0,-49.0,-13.0,643.0,104.0,924.0,-717.0,62.0,-691.0,63.0,-268.0,937.0,217.0,-249.0,29.0,-141.0,-469.0,-571.0,292.0,-314.0,-84.0,-624.0,-711.0,761.0,-63.0,583.0,254.0,101.0,-205.0,-156.0,631.0,-163.0,655.0,-316.0,-302.0,-617.0,-955.0,-264.0,23.0,-29.0,36.0,24.0
1,-150.0,420.0,939.0,-177.0,1008.0,103.0,-207.0,939.0,316.0,9.0,-605.0,835.0,897.0,-136.0,-473.0,-38.0,989.0,-160.0,346.0,889.0,-215.0,-609.0,-773.0,-202.0,-255.0,509.0,934.0,-878.0,-50.0,515.0,130.0,-179.0,995.0,-991.0,-157.0,43.0,48.0,-85.0,1.0,644.0,107.0,922.0,-718.0,61.0,-690.0,63.0,-273.0,925.0,214.0,-344.0,45.0,-134.0,-469.0,-578.0,285.0,-316.0,-80.0,-620.0,-714.0,759.0,-62.0,590.0,321.0,184.0,-242.0,-160.0,629.0,-169.0,655.0,-309.0,-304.0,-619.0,-955.0,-265.0,19.0,-31.0,47.0,3.0
2,-160.0,432.0,941.0,-162.0,982.0,98.0,-198.0,936.0,315.0,-10.0,-580.0,802.0,902.0,-139.0,-454.0,-19.0,992.0,-170.0,345.0,898.0,-200.0,-606.0,-753.0,-116.0,-272.0,492.0,919.0,-864.0,-56.0,540.0,122.0,-162.0,946.0,-988.0,-171.0,41.0,68.0,-68.0,-8.0,642.0,109.0,919.0,-719.0,60.0,-689.0,64.0,-276.0,936.0,211.0,-393.0,97.0,-195.0,-465.0,-584.0,276.0,-317.0,-74.0,-616.0,-717.0,773.0,-41.0,601.0,404.0,191.0,-311.0,-165.0,626.0,-178.0,655.0,-302.0,-308.0,-621.0,-966.0,-270.0,10.0,-38.0,20.0,0.0
3,-171.0,432.0,911.0,-174.0,999.0,115.0,-187.0,918.0,338.0,34.0,-579.0,849.0,910.0,-151.0,-460.0,-18.0,1015.0,-185.0,345.0,888.0,-220.0,-638.0,-718.0,-354.0,-275.0,517.0,919.0,-847.0,-66.0,546.0,145.0,-144.0,939.0,-994.0,-159.0,42.0,32.0,-95.0,4.0,640.0,114.0,915.0,-720.0,60.0,-688.0,65.0,-272.0,926.0,231.0,-446.0,75.0,-199.0,-464.0,-593.0,265.0,-319.0,-69.0,-611.0,-721.0,761.0,-13.0,598.0,450.0,203.0,-283.0,-173.0,620.0,-187.0,655.0,-293.0,-312.0,-622.0,-964.0,-269.0,14.0,-51.0,33.0,-1.0
4,-171.0,NaN,929.0,-189.0,1004.0,104.0,-198.0,939.0,350.0,60.0,-584.0,843.0,917.0,-151.0,-463.0,-12.0,996.0,-209.0,345.0,886.0,-204.0,-723.0,-679.0,-220.0,-271.0,525.0,923.0,-862.0,-70.0,531.0,153.0,-167.0,970.0,-1002.0,-161.0,42.0,58.0,-145.0,17.0,640.0,116.0,910.0,-722.0,59.0,-687.0,65.0,-293.0,897.0,221.0,-510.0,142.0,-238.0,-460.0,-600.0,251.0,-320.0,-61.0,-607.0,-725.0,741.0,-11.0,612.0,552.0,179.0,-334.0,-178.0,617.0,-199.0,655.0,-284.0,-318.0,-624.0,-966.0,-262.0,24.0,-40.0,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

#### Feature.34, Feature.35 and feature.36 has been dropped from the dataset

In [15]:
dataA_updated.isnull()

,fea.1,fea.2,fea.3,fea.4,fea.5,fea.6,fea.7,fea.8,fea.9,fea.10,fea.11,fea.12,fea.13,fea.14,fea.15,fea.16,fea.17,fea.18,fea.19,fea.20,fea.21,fea.22,fea.23,fea.24,fea.25,fea.26,fea.27,fea.28,fea.29,fea.30,fea.31,fea.32,fea.33,fea.37,fea.38,fea.39,fea.40,fea.41,fea.42,fea.43,fea.44,fea.45,fea.46,fea.47,fea.48,fea.49,fea.50,fea.51,fea.52,fea.53,fea.54,fea.55,fea.56,fea.57,fea.58,fea.59,fea.60,fea.61,fea.62,fea.63,fea.64,fea.65,fea.66,fea.67,fea.68,fea.69,fea.70,fea.71,fea.72,fea.73,fea.74,fea.75,fea.76,fea.77,fea.78,fea.79,fea.80,fea.81
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18995,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
18996,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
18997

In [16]:
dataA_updated.isnull().sum()

fea.1     1187
fea.2     1188
fea.3     1187
fea.4      800
fea.5      800
          ... 
fea.77     773
fea.78     773
fea.79     773
fea.80     773
fea.81     773
Length: 78, dtype: int64

#### The dataset still contains missing values in the other Feature sets. So we will apply Imputation of missing values learned during lectures. 

# Imputing Missing Values
### 1. Replace missing data by Mean values for each columns

In [17]:
dataA_updated.fillna(dataA_updated.mean(), inplace=True)

In [18]:
dataA_updated.describe()

,fea.1,fea.2,fea.3,fea.4,fea.5,fea.6,fea.7,fea.8,fea.9,fea.10,fea.11,fea.12,fea.13,fea.14,fea.15,fea.16,fea.17,fea.18,fea.19,fea.20,fea.21,fea.22,fea.23,fea.24,fea.25,fea.26,fea.27,fea.28,fea.29,fea.30,fea.31,fea.32,fea.33,fea.37,fea.38,fea.39,fea.40,fea.41,fea.42,fea.43,fea.44,fea.45,fea.46,fea.47,fea.48,fea.49,fea.50,fea.51,fea.52,fea.53,fea.54,fea.55,fea.56,fea.57,fea.58,fea.59,fea.60,fea.61,fea.62,fea.63,fea.64,fea.65,fea.66,fea.67,fea.68,fea.69,fea.70,fea.71,fea.72,fea.73,fea.74,fea.75,fea.76,fea.77,fea.78,fea.79,fea.80,fea.81
count,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.00000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000,19000.000000
mean,-132.812384,698.264485,597.541402,-307.128462,909.548077,-32.760824,61.974363,899.313498,81.650478,356.638752,330.971569,584.833130,583.589861,421.624290,-38.719833,71.807768,780.358257,-461.605043,274.476843,893.086451,-89.299933,-622.139334,233.764044,-131.400068,-360.661825,875.943278,269.614934,-570.500166,432.334420,427.547277,-124.834707,369.141625,646.348062,-862.982169,-209.995885,209.005047,-0.557360,-7.67389,-9.025347,489.460306,83.230153,209.654194,330.984199,-66.071981,290.883086,6.278762,-817.288473,140.022110,294.320568,13.456027,-9.604268,-8.376420,279.558786,-267.420091,-105.234158,204.501070,-135.484611,197.207714,299.903166,-200.212322,555.226532,443.874637,1.923355,1.402535,-3.169364,-70.556537,-90.936080,-227.353157,-124.658035,-37.973391,137.400176,374.762934,-880.583804,-47.607780,137.641192,-18.099523,4.671257,20.726834
std,275.162567,363.737566,384.063939,179.254138,189.835729,248.595835,309.776406,192.105395,320.034742,334.377778,529.666403,274.889595,298.989418,528.066821,482.015391,253.440945,285.451810,255.698513,289.947729,210.202643,214.959699,346.464727,566.656792,443.772537,218.011487,247.719619,362.563895,300.011889,485.647768,199.608760,360.339327,510.702924,280.263755,200.240826,247.087786,282.195239,609.221391,321.89966,243.393114,337.356480,359.269909,540.702160,510.794201,315.682354,442.765076,460.717761,235.677877,400.814932,224.884294,821.286196,507.493555,549.267565,387.868232,539.669067,377.318332,451.005102,330.191307,412.505140,543.541861,561.706785,332.796796,292.749308,999.512103,686.186871,849.317790,568.086872,470.279891,371.267609,471.596186,348.527414,345.537080,571.793219,213.160775,365.396483,243.870787,762.023864,470.897814,445.805043
min,-2724.000000,-855.000000,-2196.000000,-1365.000000,-245.000000,-920.000000,-1580.000000,-149.000000,-1624.000000,-1792.000000,-1545.000000,-1079.000000,-1710.000000,-1120.000000,-1634.000000,-1089.000000,-289.000000,-1379.000000,-2269.000000,-786.000000,-1944.000000,-3172.000000,-2324.000000,-2734.000000,-1873.000000,-442.000000,-1430.000000,-1933.000000,-1103.000000,-765.000000,-1717.000000,-1234.000000,-784.000000,-1578.000000,-887.000000,-1067.000000,-3322.000000,-2764.00000,-1293.000000,-767.000000,-985.000000,-975.000000,-876.000000,-705.000000,-702.000000,-983.000000,-1884.000000,-1516.000000,-1671.000000,-12310.000000,-5226.000000,-3541.000000,-924.000000,-1237.000000,-990.000000,-912.000000,-761.00000

In [18]:
# Check for number of missing values in the entire dataset
dataA_updated.isnull().sum().sum()

0

#### Using Mean Imputation we have replaced all the missing values in the dataset as we can get from the above code